# BlockSci Setup

In [ ]:
from blocksci import *
import matplotlib.pyplot as plt
import collections
import pandas as pd
import numpy as np
%matplotlib notebook

In [ ]:
chain = Blockchain("/home/ubuntu/bitcoin-data")

# Measuring different types of address use

In [ ]:
net_coins_per_block = [block.net_address_type_value() for block in chain]

In [ ]:
df = pd.DataFrame(net_coins_per_block).fillna(0).cumsum()/1e8
df = chain.heights_to_dates(df)
df = df.rename(columns={t:address_type.pretty_name(t) for t in df.columns})
df.resample("W").mean().plot()

# Transaction Fees

In [ ]:
df = pd.DataFrame(chain[465100].fees_per_byte, columns=["Satoshis per byte"])
ax = df.reset_index().plot.scatter(x="index", y="Satoshis per byte")
plt.show()

# Identifying Non-Currency Applications

In [ ]:
%time txes = chain.script_type_txes(0, len(chain), nulldata)

In [ ]:
df = pd.DataFrame([(tx.block.time, label_application(tx)) for tx in txes], columns=["date", "label"])
df2 = df.reset_index().groupby(["date", "label"]).count().unstack(level=-1).fillna(0)
df2.columns = df2.columns.droplevel()

In [ ]:
important_columns = list(df2[df2.index > pd.to_datetime("1-1-2016")].sum().sort_values()[-10:].index)
important_columns = [x for x in important_columns if "Address" not in x]
df2[df2.index > pd.to_datetime("1-1-2016")].cumsum().resample("w").mean()[important_columns].plot()

# Average Fees per Block in July in USD

In [ ]:
fees = [(block.time, np.sum(block.fees)) for block in chain.range('Jul 2017')]

In [ ]:
converter = CurrencyConverter()

In [ ]:
df = pd.DataFrame(fees, columns=["date", "fee"])
df.index = df["date"]
del df["date"]
df = satoshi_to_currency(chain, converter, df)
df.resample("d").mean().plot()

# High Fee Transactions

In [ ]:
high_fee_txes = chain.cpp.filter_tx("fee(tx) > 10000000", 0, len(chain))

In [ ]:
df = pd.DataFrame([(tx.block.height, tx.fee) for tx in high_fee_txes], columns=["height", "fee"])
df.index = df["height"]
df["fee"] = df.apply(lambda x: converter.satoshi_to_currency(x["fee"], chain[x["height"]].time), axis=1)
ax = df.plot.scatter(x="height", y="fee")